In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import  Keys 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.wait import WebDriverWait 
from selenium.webdriver.common.by import  By 
from selenium.webdriver.support import  expected_conditions as EC 

import pandas as pd
import numpy as np
import time
import datetime

In [2]:
#建立数据库
columns=['时间','发布日期','作者名','作者类型','粉丝数','投稿数','视频名','分区','子分区','视频类型','自制or转载','标签','视频长度','播放量','点赞数','投币数','收藏数','转发数','评论数','弹幕数','有无分集']
channel_yinyue_zonghe=pd.DataFrame(columns=columns)
channel_yinyue_zonghe

,时间,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,...,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集


In [3]:
driver=webdriver.Chrome()
driver.get('https://passport.bilibili.com/login') 

In [4]:
#登录
driver.find_element_by_xpath('//*[@id="login-username"]').send_keys('15717506191')
driver.find_element_by_xpath('//*[@id="login-passwd"]').send_keys('xy000000')
driver.find_element_by_xpath('//*[@id="geetest-wrap"]/div/div[5]/a[1]').click()
#需要手动过验证码

In [5]:
driver.find_element_by_xpath('//*[@id="primaryChannelMenu"]/span[3]/div/a/span').click()  #选择“音乐”频道

In [6]:
#获得分区名
channel=driver.find_element_by_xpath('//*[@id="primaryChannelMenu"]/span[3]/div/a/span').text
channel=channel.strip('0123456789+')
channel

'音乐'

In [7]:
driver.find_element_by_xpath('//*[@id="subnav"]/ul/li[9]/a').click()  #选择“特摄”子频道
sub_channel=driver.find_element_by_xpath('//*[@id="subnav"]/ul/li[9]/a').text
sub_channel

'音乐综合'

In [8]:
driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')        
driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[1]/ul/a[2]/li').click() #让视频按热度排序

In [9]:
print(datetime.datetime.now())

for month in range(1,11): 
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div/span').click()
    period=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div[2]/div[2]/div/div[2]/a[%d]'%month).text
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div[2]/div[2]/div/div[2]/a[%d]'%month).click()
    WebDriverWait(driver,60).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/ul/li[1]/div/div[2]/div[3]/a'))) 
    for page in range(5):    #爬取每个频道1月到10月每个月播放量前100的视频

        video_loc=1
        while video_loc<21:

            ### 判断作者类型###
            
            
            driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[3]/a'%video_loc).click()
            driver.switch_to_window(driver.window_handles[-1])
            
            
            author_web_show='No'
            try:
                i=0
                while author_web_show=='No':
                    driver.refresh()
                    i+=1
                    try:
                        WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="n-fs"]')))  #看作者名是否出现
                        WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="navigator"]/div/div[1]/div[1]/a[3]/span[3]')))  #看投稿数是否出现
                        author_web_show='Yes'
                    except:
                        author_web_show='No' 
                        if i>3:  
                            raise Exception  #刷新几次后仍爬不了说明用户已注销

                try:
                    driver.find_element_by_xpath('//*[@class="user-auth-subscript avatar-m organization-auth"]')
                    author_type='机构认证'

                except:
                    try:
                        driver.find_element_by_xpath('//*[@class="user-auth-subscript avatar-m personal-auth"]')
                        author_type='个人认证'
                    except:
                        author_type='无'

                fans_num=driver.find_element_by_xpath('//*[@id="n-fs"]').text  #获取作者粉丝数

                ### 获取作者投稿数 ###
                driver.find_element_by_xpath('//*[@class="n-num"]').click()
                WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="page-video"]/div/div[1]/div/div/ul/li[1]/span')))  
                video_num=driver.find_element_by_xpath('//*[@id="page-video"]/div/div[1]/div/div/ul/li[1]/span').text

                driver.close()
                driver.switch_to_window(driver.window_handles[-1])


                view_num=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[2]/span[1]/span'%video_loc).text #获取播放量
                real_dan_mu_num=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[2]/span[2]/span'%video_loc).text #获取弹幕数
                author=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[3]/a'%video_loc).text #获取作者名
                driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/a'%video_loc).click() #进入视频界面  
                driver.switch_to_window(driver.window_handles[-1]) 

                try:
                    WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="viewbox_report"]/h1/span')))  #若出现视频被删除的情况则退出

                    video_web_show='No'
                    while video_web_show=='No':
                        driver.refresh()

                        try:
                            WebDriverWait(driver,60).until(EC.presence_of_element_located((By.XPATH,'//*[@class="bilibili-player-video-time-total"]'))) #以视频长度出现时间判定网页是否加载完
                            video_web_show='Yes'

                            try:      
                                driver.find_element_by_xpath('//*[@id="multi_page"]')   #测试是否有分集视频
                                multi='是'

                            except:
                                multi='否'

                            driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')        
                            driver.execute_script('window.scrollTo(document.body.scrollHeight,0)') #再调回最上面是因为，视频长度的元素必须在屏幕内才可以被提取，否则为空

                            bv=driver.current_url[33:44] # 获取BV号（BV号在网址中）
                            print('开始BV号为%s视频属性的抓取'%bv)
                            video_name=driver.find_element_by_xpath('//*[@id="viewbox_report"]/h1/span').text #获取视频名
                            
                            ### 获取发布日期 ###
                            release_date=driver.find_element_by_xpath('//*[@id="viewbox_report"]/div/span[3]').text
                            
                            ### 获取视频类型 ###
                            try:
                                video_type=driver.find_element_by_xpath('//*[@class="activity"]').text
                                try:
                                    driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[2]/div[1]/div[1]/span[1]')
                                    video_type=video_type+'和合作视频'
                                except:
                                    pass
                            except:
                                try:
                                    driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[2]/div[1]/div[1]/span[1]')
                                    video_type='合作视频'
                                except:
                                    video_type='普通视频'


                            ### 获取点赞数 ###
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[1]').text=='点赞':
                                like_num=0
                            else:
                                like_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[1]').text 

                            ### 获取投币数 ###
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').text=='投币':
                                coin_num=0
                            else:
                                coin_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').text

                            ### 获取收藏数 ###             
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[3]').text=='收藏':
                                collect_num=0
                            else:
                                collect_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[3]').text

                            ### 获取转发数 ###       
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[4]').text=='分享':
                                forward_num=0
                            else:
                                forward_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[4]').text

                            ### 获取视频长度 ###

                            video_length=driver.find_element_by_xpath('//*[@class="bilibili-player-video-time-total"]').text  

                            ### 判断自制还是转载 ###
                            driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').click()
                            try:
                                driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[3]/div/div/div[2]/div[2]').text
                                self_made='自制'
                            except:
                                self_made='转载'
                            driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[3]/div/div/i').click()

                            ### 获取标签 ###
                            k=1
                            labels=np.array
                            while True:
                                try:
                                    labels=np.append(labels,driver.find_element_by_xpath('//*[@id="v_tag"]/ul/li[%d]'%k).text)
                                    k+=1
                                except: 
                                    break
                            labels=labels[1:]

                            ### 获取评论数 ###
                            driver.execute_script('window.scrollTo(0, 700)')        
                            WebDriverWait(driver,20).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[1]/span[2]'))) #保证"评论"二字能加载出来                                                   
                            WebDriverWait(driver,20).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[2]/div/div[4]/div')))
                            if driver.find_element_by_xpath('//*[@id="comment"]/div/div[2]/div/div[4]/div').text=='没有更多信息':    #应对评论区被关闭的情况
                                comment_num='NaN'
                            else:
                                while driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text=='':
                                    try:
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[2]/div/div[4]/div[1]')))
                                        while driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text=='':
                                            time.sleep(0.1) #很奇怪，评论数可以定位的一瞬间并不能爬取，会返回空
                                        comment_num=driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text     
                                    except:
                                        driver.refresh()    #有的时候“评论”二字出现，评论数不出现，页面好像卡住了
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="viewbox_report"]/h1/span')))
                                        driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[1]/span[1]')))

                                comment_num=driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text

                            channel_yinyue_zonghe.loc[bv]=[period,release_date,author,author_type,fans_num,video_num,video_name,channel,sub_channel,video_type,self_made,labels,video_length,view_num,like_num,coin_num,collect_num,forward_num,comment_num,real_dan_mu_num,multi]

                            print('BV号为%s的视频属性抓取结束'%bv)

                            driver.close()
                            driver.switch_to_window(driver.window_handles[-1])

                        except:
                            video_web_show='No' 
                except:
                    driver.close()
                    driver.switch_to_window(driver.window_handles[-1])
            except:
                driver.close()
                driver.switch_to_window(driver.window_handles[-1])
            
            video_loc+=1


        driver.find_element_by_xpath('//*[@class="nav-btn iconfont icon-arrowdown3"]').click() #点击下一页
        WebDriverWait(driver,100).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/ul/li[1]')))  #点击下一页后等页面刷新
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/div[2]/ul/li[2]').click()  #回到第1页
    WebDriverWait(driver,100).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/div[2]/ul/li[1]')))
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/div[2]/ul/li[1]').click()  #发现很多月份漏爬了约20个，而且恰好是第一页，所以再点一次第一页
        
        
print(datetime.datetime.now()) 

2020-12-20 19:34:23.064866


<ipython-input-9-e79ad032d466>:17: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])
<ipython-input-9-e79ad032d466>:54: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])
<ipython-input-9-e79ad032d466>:61: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为1mJ411G73H视频属性的抓取
BV号为1mJ411G73H的视频属性抓取结束


<ipython-input-9-e79ad032d466>:181: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为1gJ41157Gp视频属性的抓取
BV号为1gJ41157Gp的视频属性抓取结束
开始BV号为1YJ411n7VK视频属性的抓取
BV号为1YJ411n7VK的视频属性抓取结束
开始BV号为1mJ411b7Hi视频属性的抓取
BV号为1mJ411b7Hi的视频属性抓取结束
开始BV号为1gJ411H74f视频属性的抓取
BV号为1gJ411H74f的视频属性抓取结束
开始BV号为18J411V7Cg视频属性的抓取
BV号为18J411V7Cg的视频属性抓取结束
开始BV号为1LJ411G7Nw视频属性的抓取
BV号为1LJ411G7Nw的视频属性抓取结束
开始BV号为1nJ411G7db视频属性的抓取
BV号为1nJ411G7db的视频属性抓取结束
开始BV号为1mJ411G7cJ视频属性的抓取
BV号为1mJ411G7cJ的视频属性抓取结束
开始BV号为1QJ411H7mp视频属性的抓取
BV号为1QJ411H7mp的视频属性抓取结束
开始BV号为1j741167RP视频属性的抓取
BV号为1j741167RP的视频属性抓取结束
开始BV号为1y7411s7Xh视频属性的抓取
BV号为1y7411s7Xh的视频属性抓取结束
开始BV号为1TJ411C7jS视频属性的抓取
BV号为1TJ411C7jS的视频属性抓取结束
开始BV号为18J411V7nA视频属性的抓取
BV号为18J411V7nA的视频属性抓取结束
开始BV号为1UJ411J7Gh视频属性的抓取
BV号为1UJ411J7Gh的视频属性抓取结束
开始BV号为1kJ411J7iF视频属性的抓取
BV号为1kJ411J7iF的视频属性抓取结束
开始BV号为1KJ411V7qB视频属性的抓取
BV号为1KJ411V7qB的视频属性抓取结束
开始BV号为1dJ41157kQ视频属性的抓取
BV号为1dJ41157kQ的视频属性抓取结束
开始BV号为1SJ411H7Fd视频属性的抓取
BV号为1SJ411H7Fd的视频属性抓取结束
开始BV号为1e7411i7s8视频属性的抓取
BV号为1e7411i7s8的视频属性抓取结束
开始BV号为1z7411v7CU视频属性的抓取
BV号为1z7411v7CU的视频属性抓取结束
开始BV号为1g7411k7ho视频属性的抓取
BV号为1g7411k7ho的视

...

开始BV号为1uE411u7Mt视频属性的抓取
BV号为1uE411u7Mt的视频属性抓取结束
开始BV号为1q7411y7Sq视频属性的抓取
BV号为1q7411y7Sq的视频属性抓取结束
开始BV号为1c7411U7Ph视频属性的抓取
BV号为1c7411U7Ph的视频属性抓取结束
开始BV号为11Q4y1M7q3视频属性的抓取
BV号为11Q4y1M7q3的视频属性抓取结束
开始BV号为1V64y1u7DC视频属性的抓取
BV号为1V64y1u7DC的视频属性抓取结束
开始BV号为1bE411w7nf视频属性的抓取
BV号为1bE411w7nf的视频属性抓取结束
开始BV号为1cE411N7cM视频属性的抓取
BV号为1cE411N7cM的视频属性抓取结束
开始BV号为1s741117xy视频属性的抓取
BV号为1s741117xy的视频属性抓取结束
开始BV号为1gQ4y1M7MQ视频属性的抓取
BV号为1gQ4y1M7MQ的视频属性抓取结束
开始BV号为1Q7411m73y视频属性的抓取
BV号为1Q7411m73y的视频属性抓取结束
开始BV号为16K41177q2视频属性的抓取
BV号为16K41177q2的视频属性抓取结束
开始BV号为1mE411J7cm视频属性的抓取
BV号为1mE411J7cm的视频属性抓取结束
开始BV号为1tE411w71Z视频属性的抓取
BV号为1tE411w71Z的视频属性抓取结束
开始BV号为1A7411f7Ff视频属性的抓取
BV号为1A7411f7Ff的视频属性抓取结束
开始BV号为11E411A7ku视频属性的抓取
BV号为11E411A7ku的视频属性抓取结束
开始BV号为12E411H7CW视频属性的抓取
BV号为12E411H7CW的视频属性抓取结束
开始BV号为1Ha4y1t7xe视频属性的抓取
BV号为1Ha4y1t7xe的视频属性抓取结束
开始BV号为1Y741127Sy视频属性的抓取
BV号为1Y741127Sy的视频属性抓取结束
开始BV号为187411y7EM视频属性的抓取
BV号为187411y7EM的视频属性抓取结束
开始BV号为1nE411T73m视频属性的抓取
BV号为1nE411T73m的视频属性抓取结束
开始BV号为1wE411P7QG视频属性的抓取
BV号为1wE411P7QG的视

开始BV号为1B54y1X7Ls视频属性的抓取
BV号为1B54y1X7Ls的视频属性抓取结束
开始BV号为14f4y1m7oR视频属性的抓取
BV号为14f4y1m7oR的视频属性抓取结束
开始BV号为1cK411s7gM视频属性的抓取
BV号为1cK411s7gM的视频属性抓取结束
开始BV号为19i4y1t7ia视频属性的抓取
BV号为19i4y1t7ia的视频属性抓取结束
开始BV号为18t4y1C7go视频属性的抓取
BV号为18t4y1C7go的视频属性抓取结束
开始BV号为1gZ4y1s7Ms视频属性的抓取
BV号为1gZ4y1s7Ms的视频属性抓取结束
开始BV号为1HV411C7jC视频属性的抓取
BV号为1HV411C7jC的视频属性抓取结束
开始BV号为1jv411z75D视频属性的抓取
BV号为1jv411z75D的视频属性抓取结束
开始BV号为17T4y1g7At视频属性的抓取
BV号为17T4y1g7At的视频属性抓取结束
开始BV号为1DV411k7NW视频属性的抓取
BV号为1DV411k7NW的视频属性抓取结束
开始BV号为1LA411q7G8视频属性的抓取
BV号为1LA411q7G8的视频属性抓取结束
开始BV号为1Qz4y1R7XT视频属性的抓取
BV号为1Qz4y1R7XT的视频属性抓取结束
开始BV号为1N5411s7jp视频属性的抓取
BV号为1N5411s7jp的视频属性抓取结束
开始BV号为1N5411x72Z视频属性的抓取
BV号为1N5411x72Z的视频属性抓取结束
开始BV号为19i4y1x7u3视频属性的抓取
BV号为19i4y1x7u3的视频属性抓取结束
开始BV号为1QC4y1H78n视频属性的抓取
开始BV号为1QC4y1H78n视频属性的抓取
BV号为1QC4y1H78n的视频属性抓取结束
开始BV号为1Qi4y1x7Gm视频属性的抓取
BV号为1Qi4y1x7Gm的视频属性抓取结束
开始BV号为1jT4y137Ut视频属性的抓取
BV号为1jT4y137Ut的视频属性抓取结束
开始BV号为1HQ4y1N7XH视频属性的抓取
BV号为1HQ4y1N7XH的视频属性抓取结束
开始BV号为18A411t7D9视频属性的抓取
BV号为18A411t7D9的视频属性抓取结束
开始BV号为1ff4y117j8

开始BV号为11f4y1y7et视频属性的抓取
BV号为11f4y1y7et的视频属性抓取结束
开始BV号为1oz4y1978U视频属性的抓取
BV号为1oz4y1978U的视频属性抓取结束
开始BV号为1pv41167Jx视频属性的抓取
BV号为1pv41167Jx的视频属性抓取结束
开始BV号为1tZ4y1W7t7视频属性的抓取
BV号为1tZ4y1W7t7的视频属性抓取结束
开始BV号为1Nv411B78c视频属性的抓取
BV号为1Nv411B78c的视频属性抓取结束
开始BV号为1Lg4y1q7TX视频属性的抓取
BV号为1Lg4y1q7TX的视频属性抓取结束
开始BV号为1Qf4y1y7Tx视频属性的抓取
BV号为1Qf4y1y7Tx的视频属性抓取结束
开始BV号为1x54y1B7Ey视频属性的抓取
BV号为1x54y1B7Ey的视频属性抓取结束
开始BV号为1zv411B7PR视频属性的抓取
BV号为1zv411B7PR的视频属性抓取结束
开始BV号为1Qf4y127R8视频属性的抓取
BV号为1Qf4y127R8的视频属性抓取结束
开始BV号为1c5411W7vS视频属性的抓取
BV号为1c5411W7vS的视频属性抓取结束
开始BV号为1FA411B7sC视频属性的抓取
BV号为1FA411B7sC的视频属性抓取结束
开始BV号为1Y54y1Q7rT视频属性的抓取
BV号为1Y54y1Q7rT的视频属性抓取结束
开始BV号为1Vz411i7sR视频属性的抓取
BV号为1Vz411i7sR的视频属性抓取结束
开始BV号为1Qk4y1z7sc视频属性的抓取
BV号为1Qk4y1z7sc的视频属性抓取结束
开始BV号为1ip4y1D7iY视频属性的抓取
BV号为1ip4y1D7iY的视频属性抓取结束
开始BV号为1z5411p7ga视频属性的抓取
BV号为1z5411p7ga的视频属性抓取结束
开始BV号为1Ek4y1z7Wf视频属性的抓取
BV号为1Ek4y1z7Wf的视频属性抓取结束
开始BV号为1PZ4y1H7cR视频属性的抓取
开始BV号为1PZ4y1H7cR视频属性的抓取
BV号为1PZ4y1H7cR的视频属性抓取结束
开始BV号为15v411671T视频属性的抓取
BV号为15v411671T的视频属性抓取结束
开始BV号为1sK411s7WN

BV号为1kZ4y1M7h4的视频属性抓取结束
开始BV号为1iA411n7mh视频属性的抓取
BV号为1iA411n7mh的视频属性抓取结束
开始BV号为1vv411q7Mw视频属性的抓取
BV号为1vv411q7Mw的视频属性抓取结束
开始BV号为1v54y1m7Y8视频属性的抓取
BV号为1v54y1m7Y8的视频属性抓取结束
开始BV号为1uA411e7i1视频属性的抓取
BV号为1uA411e7i1的视频属性抓取结束
开始BV号为1C54y1i72B视频属性的抓取
BV号为1C54y1i72B的视频属性抓取结束
开始BV号为14K4y1e73V视频属性的抓取
BV号为14K4y1e73V的视频属性抓取结束
开始BV号为185411b7tR视频属性的抓取
BV号为185411b7tR的视频属性抓取结束
开始BV号为1ht4y1D7p7视频属性的抓取
BV号为1ht4y1D7p7的视频属性抓取结束
开始BV号为1th411d7qJ视频属性的抓取
BV号为1th411d7qJ的视频属性抓取结束
开始BV号为1ch411d75i视频属性的抓取
BV号为1ch411d75i的视频属性抓取结束
开始BV号为1sa4y177xH视频属性的抓取
BV号为1sa4y177xH的视频属性抓取结束
开始BV号为1Dk4y1m7kz视频属性的抓取
BV号为1Dk4y1m7kz的视频属性抓取结束
开始BV号为1f541187AB视频属性的抓取
BV号为1f541187AB的视频属性抓取结束
开始BV号为1tt4y1D7ih视频属性的抓取
BV号为1tt4y1D7ih的视频属性抓取结束
开始BV号为1gh411R7ox视频属性的抓取
BV号为1gh411R7ox的视频属性抓取结束
开始BV号为1B541187gU视频属性的抓取
BV号为1B541187gU的视频属性抓取结束
开始BV号为1GA411n7KX视频属性的抓取
BV号为1GA411n7KX的视频属性抓取结束
开始BV号为1pV411U7Hh视频属性的抓取
BV号为1pV411U7Hh的视频属性抓取结束
开始BV号为1C64y1F7XF视频属性的抓取
BV号为1C64y1F7XF的视频属性抓取结束
开始BV号为1kf4y1X7W9视频属性的抓取
BV号为1kf4y1X7W9的视频属性抓取结束
开始BV号为1GC4y187yJ

BV号为1Zr4y1c71h的视频属性抓取结束
开始BV号为12z4y1Z7zZ视频属性的抓取
BV号为12z4y1Z7zZ的视频属性抓取结束
开始BV号为1da4y1W7J3视频属性的抓取
BV号为1da4y1W7J3的视频属性抓取结束
开始BV号为1vV411y7Gd视频属性的抓取
BV号为1vV411y7Gd的视频属性抓取结束
开始BV号为1Er4y1w7x4视频属性的抓取
BV号为1Er4y1w7x4的视频属性抓取结束
开始BV号为1fk4y1C7vt视频属性的抓取
BV号为1fk4y1C7vt的视频属性抓取结束
开始BV号为1b54y117ko视频属性的抓取
BV号为1b54y117ko的视频属性抓取结束
开始BV号为1rK4y177qW视频属性的抓取
BV号为1rK4y177qW的视频属性抓取结束
开始BV号为14i4y1E766视频属性的抓取
BV号为14i4y1E766的视频属性抓取结束
开始BV号为17t4y1i7i5视频属性的抓取
BV号为17t4y1i7i5的视频属性抓取结束
开始BV号为1GZ4y157pn视频属性的抓取
BV号为1GZ4y157pn的视频属性抓取结束
开始BV号为1gV411273j视频属性的抓取
BV号为1gV411273j的视频属性抓取结束
开始BV号为1TT4y1F73A视频属性的抓取
BV号为1TT4y1F73A的视频属性抓取结束
开始BV号为1qD4y1d7Wt视频属性的抓取
BV号为1qD4y1d7Wt的视频属性抓取结束
开始BV号为1vV41117gy视频属性的抓取
BV号为1vV41117gy的视频属性抓取结束
开始BV号为1gt4y1i7zi视频属性的抓取
BV号为1gt4y1i7zi的视频属性抓取结束
开始BV号为1Et4y1v7pf视频属性的抓取
BV号为1Et4y1v7pf的视频属性抓取结束
开始BV号为12A41177MA视频属性的抓取
BV号为12A41177MA的视频属性抓取结束
开始BV号为1554y1r7ch视频属性的抓取
BV号为1554y1r7ch的视频属性抓取结束
开始BV号为17K411A7U3视频属性的抓取
BV号为17K411A7U3的视频属性抓取结束
开始BV号为1EV411279W视频属性的抓取
BV号为1EV411279W的视频属性抓取结束
开始BV号为1x5411j7qd

In [13]:
driver.switch_to_window(driver.window_handles[-1])

<ipython-input-13-dd0a1399d54d>:1: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


In [10]:
channel_yinyue_zonghe.groupby('时间').agg('count')

,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,自制or转载,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集
时间,,,,,,,,,,,,,,,,,,,,
10月,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99
1月,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99
2月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
3月,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99
4月,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98
5月,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99
6月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
7月,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98
8月,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99


In [11]:
channel_yinyue_zonghe.to_json('channel_yinyue_zonghe.json')